In [1]:
##importing libraries that are need to support analysis
import pandas as pd
import numpy as np
import plotly.express as px
from ebmdatalab import bq, maps, charts
import matplotlib.pyplot as plt
import os

figures_path = os.path.join('..','figures/')

In [2]:
df = pd.read_csv(os.path.join("..","regression", "data_for_stata.csv"))
df.head()

,practice,pct,"('items', 0)","('items', 1)","('denominator', '')","('rate', '')",LSOA11NM,ruc11cd,RUC11,mean_percentile,...,Value_satisfaction,Value_long_term_health,Value_over_65,Value_under_18,code,dispensing_patients,total_items,list_size,ODS,supplier
0,A81001,00K,5243.0,5860.0,11103.0,0.527785,Stockton-on-Tees 014G,C,Urban city and town,0.445317,...,83.627562,49.450467,23.323544,20.264317,A81001,0.0,94138.0,4097.083333,A81001,TPP
1,A81002,00K,22703.0,38712.0,61415.0,0.630335,Stockton-on-Tees 014F,C,Urban city and town,0.493340,...,80.971049,58.535567,21.194615,18.908872,A81002,0.0,451497.0,19829.750000,A81002,TPP
2,A81003,00K,NaN,NaN,NaN,NaN,Hartlepool 007D,C,Urban city and town,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A81004,00M,10306.0,16142.0,26448.0,0.610330,Middlesbrough 015C,C,Urban city and town,0.521257,...,87.585723,50.589328,18.819479,18.937531,A81004,0.0,214788.0,10307.333333,A81004,TPP
4,A81005,00M,1920.0,31762.0,33682.0,0.942996,Redcar and Cleveland 018A,C,Urban city and town,0.342573,...,84.936370,63.431023,27.956587,17.851796,A81005,0.0,195485.0,8020.416667,A81005,TPP


In [3]:

# prepare data for plotting
dfp = df.copy()[["('rate', '')", 'dispensing_patients', "supplier", "list_size"]]
dfp = dfp.rename(columns={"('rate', '')":'Proportion prescribed as 28 day durations'})

# include only practices with >100 patients
# note there are many non-standard practices with no list size that we are also excluding here
dfp = dfp.loc[dfp["list_size"]>100]



In [4]:

### Dispensing

# create binary for dispensing practice
dfp['dispensing_practice'] = "no"
dfp.loc[pd.isnull(dfp['dispensing_patients']),'dispensing_practice'] = "unknown"
dfp.loc[dfp['dispensing_patients']>=1,'dispensing_practice'] = "yes"

print (dfp.groupby("dispensing_practice")["dispensing_practice"].count())

## reassign unknowns to "no"
dfp.loc[dfp['dispensing_practice']=="unknown",'dispensing_practice'] = "no"

# plot data
fig = px.histogram(dfp, x="Proportion prescribed as 28 day durations", nbins=20, color="dispensing_practice",
barmode= "overlay")

fig.update_layout(
    title="Proportion prescribed as 28 day durations of all 28, 56 and 84-day prescriptions <br> by dispensing status")

fig.write_image(figures_path+"histogram_dispensing.png", format="png", scale=2)
fig.show()

dispensing_practice
no         6234
unknown      12
yes        1010
Name: dispensing_practice, dtype: int64


ValueError: 
The orca executable is required to export figures as static images,
but it could not be found on the system path.

Searched for executable 'orca' on the following path:
    c:\Users\hcurtis\Anaconda3\lib\site-packages\shapely\DLLs
    c:\Users\hcurtis\Anaconda3\DLLs
    c:\Users\hcurtis\Anaconda3\Library\bin
    c:\Users\hcurtis\Anaconda3
    C:\Users\hcurtis\Anaconda3
    C:\Users\hcurtis\Anaconda3\Library\mingw-w64\bin
    C:\Users\hcurtis\Anaconda3\Library\usr\bin
    C:\Users\hcurtis\Anaconda3\Library\bin
    C:\Users\hcurtis\Anaconda3\Scripts
    C:\Users\hcurtis\Anaconda3\bin
    C:\Users\hcurtis\Anaconda3\condabin
    C:\WINDOWS\System32\WindowsPowerShell\v1.0
    C:\ProgramData\Oracle\Java\javapath
    C:\WINDOWS\system32
    C:\WINDOWS
    C:\WINDOWS\System32\Wbem
    C:\Program Files\IBM\SPSS\Statistics\24\JRE\bin
    C:\Program Files (x86)\Novell\ZENworks\bin
    C:\Program Files\Novell\iPrint
    C:\Program Files (x86)\Git\cmd
    C:\Users\hcurtis\AppData\Local

In [ ]:
### Software supplier

dfp2 = dfp.copy()

# Prepare data - inspect missing suppliers
dfp2["supplier"] = dfp2["supplier"].fillna("Unknown")
print (dfp2.groupby("supplier")["dispensing_practice"].count())
# drop unknowns as these may be non-standard practices or their IDs were 
# simply misaligned with the software file - either way we have no way to categorise them.
dfp2 = dfp2.loc[dfp2["supplier"] != "Unknown"]

fig = px.histogram(dfp2, x="Proportion prescribed as 28 day durations", nbins=15, 
                   color="supplier",
                    #barmode= "overlay",
                    facet_col='supplier',
                    facet_col_wrap=2
                    )


fig.update_layout(
    title="Proportion prescribed as 28 day durations of all 28, 56 and 84-day prescriptions <br> by software supplier",
    margin=dict(t=100)
    )

fig.write_image(figures_path+"histogram_supplier.png", format="png", scale=2)
fig.show()

supplier
EMIS         4000
Microtest      37
TPP          2611
Unknown       344
Vision        264
Name: dispensing_practice, dtype: int64
